# Thinking1 对于同一个带约束的规划问题，LP和MIP哪个运算复杂度高，为什么？
            答：MIP(混合整数规划混合整数规划)要比LP(线性规划）的运算复杂度要高，运算花费的时间更长，因为单纯的线性规划，在计算最优解的时候更容易直接通过方程组计算的方式获得最终结果，算法上更容易优化然后             更快的获得结果，但是MIP因为存在部分参数是限定为整数类型的，算法在计算的过程中需要暴力穷举可能的整数点，然后依次代入求解最优解，更加耗时。
# Thinking2 TSP与VRP问题的关系是怎样的？
            答：VRP可以看成是TSP的推广，TSP旅行商问题是求解一辆车如何能够用最短的时候走最短的路径去满足分布在不同地点的客户需求的问题，VRP车辆路径问题与TSP相同的是也具有众多数量的客户需要满足，不同的是               VRP求解的是多辆车同时去满足这些用户，VRP问题更加契合现实生活中的大部分问题，计算起来也更加复杂。



In [1]:
import pandas as pd
data=pd.read_csv("G://python_lesson//L20//L22//santa//family_data.csv",index_col='family_id')
data

,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
family_id,,,,,,,,,,,
0,52,38,12,82,33,75,64,76,10,28,4
1,26,4,82,5,11,47,38,6,66,61,4
2,100,54,25,12,27,82,10,89,80,33,3
3,2,95,1,96,32,6,40,31,9,59,2
4,53,1,47,93,26,3,46,16,42,39,4
...,...,...,...,...,...,...,...,...,...,...,...
4995,16,1,66,33,18,70,56,46,86,60,4
4996,88,66,20,17,26,54,81,91,59,48,2
4997,32,66,54,17,27,21,74,81,3,7,6


In [2]:
#preference cost 矩阵 pcost_mat
#accounting cost 矩阵 acost_mat
#家庭人数 FAMILY_SIZE
#每个家庭的倾向性选择（choice_）DESIRED
def get_penalty(n,choice):
    penalty=None
    if choice==0:
        penalty=0
    if choice==1:
        penalty=50
    if choice==2:
        penalty=50+9*n
    if choice==3:
        penalty=100+9*n
    if choice==4:
        penalty=200+9*n
    if choice==5:
        penalty=200+18*n
    if choice==6:
        penalty=300+18*n
    if choice==7:
        penalty=300+36*n
    if choice==8:
        penalty=400+36*n
    if choice==9:
        penalty=500+(36+199)*n
    if choice>9:
        penalty=500+(36+398)*n
    return penalty
        

In [3]:
import numpy as np
N_DAYS=100
N_FAMILY=5000
MIN_OCCUPANCY=125
MAX_OCCUPANCY=300

pcost_mat=np.full(shape=(N_FAMILY,N_DAYS),fill_value=999999)

for f in range(N_FAMILY):
    #f家庭的成员数
    f_num=data.loc[f,'n_people']
    #先把所有的数初始化为choice为10的penalty
    pcost_mat[f,:]=get_penalty(f_num,10)
    for choice in range(10):
        temp=data.loc[f][choice]
        penalty=get_penalty(f_num,choice)
        pcost_mat[f,temp-1]=penalty

pcost_mat

array([[2236, 2236, 2236, ..., 2236, 2236, 2236],
       [2236, 2236, 2236, ..., 2236, 2236, 2236],
       [1802, 1802, 1802, ..., 1802, 1802,    0],
       ...,
       [3104, 3104,  616, ..., 3104, 3104, 3104],
       [ 390, 2670, 2670, ..., 2670, 2670, 2670],
       [2236, 2236, 2236, ..., 2236, 2236, 2236]])

In [4]:
pcost_mat.shape

(5000, 100)

In [5]:
#accounting penalty 前一天和当天不同参观人数组合下的清洁费用
acost_mat=np.zeros(shape=(MAX_OCCUPANCY+1,MAX_OCCUPANCY+1),dtype=np.float64)
for i in range(acost_mat.shape[0]):
    for j in range(acost_mat.shape[1]):
        diff=abs(i-j)
        acost_mat[i,j]=max(0,(i-125)/400*i**(0.5+diff/50.0))
acost_mat        
        


array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [4.16316072e+15, 3.71482922e+15, 3.31477861e+15, ...,
        7.46610759e+00, 8.36716954e+00, 9.37697794e+00],
       [4.79555148e+15, 4.27883100e+15, 3.81778713e+15, ...,
        8.43020770e+00, 7.52185316e+00, 8.43020770e+00],
       [5.52415954e+15, 4.92860244e+15, 4.39725208e+15, ...,
        9.51970597e+00, 8.49339085e+00, 7.57772228e+00]])

In [6]:
FAMILY_SIZE=data['n_people'].values
FAMILY_SIZE

array([4, 4, 3, ..., 6, 5, 4], dtype=int64)

In [7]:
#每个家庭的期望访问日期
DESIRED=data.values[:,:-1]-1
DESIRED

array([[51, 37, 11, ..., 75,  9, 27],
       [25,  3, 81, ...,  5, 65, 60],
       [99, 53, 24, ..., 88, 79, 32],
       ...,
       [31, 65, 53, ..., 80,  2,  6],
       [66, 91,  3, ..., 11, 25, 69],
       [12, 10, 24, ..., 38, 17, 46]], dtype=int64)

In [8]:
#线性规划
from ortools.linear_solver import pywraplp
def solveLP():
    solver=pywraplp.Solver('AssignmentProblem',pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)
    x={}
    candidates=[[] for x in range(N_DAYS)]
    for i in range(N_FAMILY):
        for j in DESIRED[i,:]:
            candidates[j].append(i)
            x[i,j]=solver.BoolVar('x[%i,%i]' %(i,j))
    #x[i,j]第i个家庭是否在第i天出现
    #daily_occupancy每天出现的总人数
    daily_occupancy=[solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]])\
                     for j in range(N_DAYS)]               
    family_prensence=[solver.Sum(x[i,j] for j in DESIRED[i,:]) \
                      for i in range(N_FAMILY)]
    #目标函数最小值
    preference_cost=solver.Sum([pcost_mat[i,j]*x[i,j] for i in range(N_FAMILY) \
                                for j in DESIRED[i,:]])
    solver.Minimize(preference_cost)
    #任意两天出现的人数差值不要太大
    for j in range(N_DAYS-1):
        solver.Add(daily_occupancy[j]-daily_occupancy[j+1]<=25)
        solver.Add(daily_occupancy[j+1]-daily_occupancy[j]<=25)
    for i in range(N_FAMILY):
        solver.Add(family_prensence[i]==1)
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j]>=MIN_OCCUPANCY)
        solver.Add(daily_occupancy[j]<=MAX_OCCUPANCY)            
    result=solver.Solve()
    temp=[(i,j,x[i,j].solution_value()) for i in range(N_FAMILY) for j in DESIRED[i,:] if x[i,j].solution_value() >0]
    df=pd.DataFrame(temp,columns=['faminly_id','day','result'])
    return df
result=solveLP()
           

In [9]:
result

,faminly_id,day,result
0,0,51,1.0
1,1,25,1.0
2,2,99,1.0
3,3,1,1.0
4,4,52,1.0
...,...,...,...
5068,4995,15,1.0
5069,4996,87,1.0
5070,4997,31,1.0
5071,4998,91,1.0


In [10]:
result['result'].value_counts()

1.000000    4301
1.000000     627
0.750000       2
0.666667       2
0.666667       2
            ... 
0.750000       1
0.400000       1
0.400000       1
0.400000       1
0.947917       1
Name: result, Length: 144, dtype: int64

In [11]:
#未安排的家庭
THRS=0.999
unassigned_df=result[(result.result<THRS)&(result.result>1-THRS)]
unassigned_df

,faminly_id,day,result
59,59,38,0.25
60,59,14,0.75
241,240,32,0.75
242,240,56,0.25
264,262,31,0.50
...,...,...,...
4983,4912,8,0.40
4985,4914,38,0.60
4986,4914,43,0.40
5033,4961,53,0.75


In [12]:
#计算每天到访总人数
assigened_df=result[result.result>THRS]
assigened_df['family_size']=FAMILY_SIZE[assigened_df.faminly_id]
assigened_df

<ipython-input-12-8cd08466a26a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  assigened_df['family_size']=FAMILY_SIZE[assigened_df.faminly_id]


,faminly_id,day,result,family_size
0,0,51,1.0,4
1,1,25,1.0,4
2,2,99,1.0,3
3,3,1,1.0,2
4,4,52,1.0,4
...,...,...,...,...
5068,4995,15,1.0,4
5069,4996,87,1.0,2
5070,4997,31,1.0,6
5071,4998,91,1.0,5


In [13]:
occupancy=assigened_df.groupby('day').family_size.sum()
occupancy

day
0     290
1     271
2     294
3     293
4     263
     ... 
95    158
96    128
97    125
98    122
99    124
Name: family_size, Length: 100, dtype: int64

In [14]:
min_occupancy=np.array([max(0,MIN_OCCUPANCY-x)for x in occupancy])
max_occupancy=np.array([MAX_OCCUPANCY-x for x in occupancy])
print(min_occupancy)
print(max_occupancy)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 4 0 0 0 0 0 2 0 0 0 0
 0 0 4 0 0 0 0 0 0 5 0 0 0 0 0 0 0 3 0 0 0 0 0 0 3 1]
[ 10  29   6   7  37  58  77  53  27   3  12   8  25  50  62  28   8   8
  29  52  77  56  36   9   8   4  27  51  66  49  22  17  48  65  95 116
  98  67  47  69  90 117  96  71  53  19  44  77  96 102  78  52  45  77
  92 115 127 104  81 102 126 159 176 179 151 130 122 142 164 177 175 165
 142 115 142 165 179 175 158 133 114 133 162 180 172 145 126  97 123 148
 170 178 170 142 122 142 172 175 178 176]


In [15]:
#用整数规划求解剩余未安排的家庭
def solveIP(families,min_occupancy,max_occupancy):
    solver=pywraplp.Solver('AssignmentProblem',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    n_families=len(families)
    
    x={}
    candidates=[[] for x in range(N_DAYS)]
    for i in families:
        for j in DESIRED[i,:]:
            candidates[j].append(i)
            x[i,j]=solver.BoolVar('x[%i,%i]' %(i,j))
    #x[i,j]第i个家庭是否在第i天出现
    #daily_occupancy每天出现的总人数
    daily_occupancy=[solver.Sum([x[i,j] * FAMILY_SIZE[i] for i in candidates[j]])\
                     for j in range(N_DAYS)]               
    family_prensence=[solver.Sum(x[i,j] for j in DESIRED[i,:]) \
                      for i in families]
    #目标函数最小值
    preference_cost=solver.Sum([pcost_mat[i,j]*x[i,j] for i in families \
                                for j in DESIRED[i,:]])
    solver.Minimize(preference_cost)

    for i in range(n_families):
        solver.Add(family_prensence[i]==1)
    for j in range(N_DAYS):
        solver.Add(daily_occupancy[j]>=min_occupancy[j])
        solver.Add(daily_occupancy[j]<=max_occupancy[j])            
    result=solver.Solve()
    
    temp=[(i,j,x[i,j].solution_value()) for i in families for j in DESIRED[i,:] if x[i,j].solution_value() >0]
    df=pd.DataFrame(temp,columns=['faminly_id','day','result'])
    return df
    


In [16]:
reslut1=solveIP(unassigned_df.faminly_id.unique(),min_occupancy,max_occupancy)
reslut1

,faminly_id,day,result
0,59,38,1.0
1,240,32,1.0
2,262,31,1.0
3,357,24,1.0
4,488,39,1.0
...,...,...,...
64,4869,59,1.0
65,4886,98,1.0
66,4912,17,1.0
67,4914,38,1.0


In [17]:
df=pd.concat((assigened_df[['faminly_id','day']],reslut1[['faminly_id','day']])).sort_values('faminly_id')
df

,faminly_id,day
0,0,51
1,1,25
2,2,99
3,3,1
4,4,52
...,...,...
5068,4995,15
5069,4996,87
5070,4997,31
5071,4998,91


In [28]:
from numba import njit
#计算当前安排的成本
@njit(fastmath=True)
def pcost(prediction):
    daily_occupancy=np.zeros(N_DAYS+1,dtype=np.int64)
    penalty=0
    for (i,p) in enumerate(prediction):
        n=FAMILY_SIZE[i]
        penalty+=pcost_mat[i,p]
        daily_occupancy[p]+=n
    return penalty,daily_occupancy
@njit(fastmath=True)
def acost(daily_occupancy):
    accounting_cost=0
    num_out_range=0
    for day in range(N_DAYS):
        n_p1=daily_occupancy[day+1]
        n=daily_occupancy[day]
        num_out_range+=(n>MAX_OCCUPANCY)or(n<MIN_OCCUPANCY)
        accounting_cost+=acost_mat[n,n_p1]
    return accounting_cost,num_out_range
@njit(fastmath=True)
def cost_function(prediction):
    penalty,daily_occupancy=pcost(prediction)
    accounting_cost,num_out_range=acost(daily_occupancy)
    final_score=penalty+accounting_cost+num_out_range*999999999
    return final_score


In [29]:
prediction=df.day.values
final_cost=cost_function(prediction)
final_cost

109212.10210277568

In [30]:
final_cost

109212.10210277568

In [31]:
prediction=df.day.values
prediction

array([51, 25, 99, ..., 31, 91, 12], dtype=int64)

In [32]:
def save_result(pred,filename):
    result=pd.DataFrame(range(N_FAMILY),columns=['faminly_id'])
    result.to_csv(filename,index=False)
    print(filename+' saved')
    return result
result=save_result(prediction,'submission.csv')

submission.csv saved


In [33]:
#打雷法，遍历每个家庭的各项选择，并替代现有的安排列表，然后计算更新之后的cost是否更小，如果更小则替换
def find_better(pred):
    fobs=np.argsort(FAMILY_SIZE)
    score=cost_function(pred)
    original_score=np.inf
    while score<original_score:
        original_score=score
        for faminly_id in fobs:
            for pick in range(10):
                day=DESIRED[faminly_id,pick]
                oldvalue=pred[faminly_id]
                pred[faminly_id]=day
                new_socre=cost_function(pred)
                if new_socre<score:
                    score=new_socre
                else:
                    pred[faminly_id]=oldvalue
        print(score,end='\r')
        
        
        

In [34]:
new=prediction.copy()
find_better(new)


In [25]:
prediction

array([51, 25, 99, ..., 31, 91, 12], dtype=int64)

In [ ]:
cost_function(prediction)